In [1]:
import pandas as pd
import os
import re
import warnings
warnings.filterwarnings('ignore')
from sqlalchemy import create_engine, text, inspect


# Function for getting the name of a DataFrame
def get_var_name(var):
    try:
        for name, value in globals().items():
            if value is var:
                return name
    except Exception as e:
        print(f'Error getting variable name: {e}')
    return None

def validate_data(df, show_counts=True):
    try:
        df_name = get_var_name(df)
        print(f'#########################################################################################################################################################################################\nDataFrame: {df_name}')
        # Snapshot the dataset
        display(df)
        # Check for unique values
        unique_counts = pd.DataFrame(df.nunique())
        unique_counts = unique_counts.reset_index().rename(columns={0:'No. of Unique Values', 'index':'Field Name'})
        print("Unique values per field:")
        pd.set_option('display.max_rows', None)
        display(unique_counts)
        pd.reset_option('display.max_rows')
        # Checking for duplicates
        duplicate_count = df.duplicated().sum()
        print("\nNumber of duplicate rows:")
        print(duplicate_count,'\n')
        info = df.info(show_counts=show_counts)
        display(info)
        # Summary stats
        print("\nSummary statistics:")
        display(df.describe())
        print('End of data validation\n#########################################################################################################################################################################################\n')
    except Exception as e:
        print(f'Error validating data: {e}')

def export_to_csv(df, **kwargs):
    try:
        # Obtaining wanted directory
        directory = kwargs.get('directory',r"C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Python CSV Repositry")
        
        # Obtaining name of DataFrame
        df_name = kwargs.get('df_name',get_var_name(df))
        if not isinstance(df_name, str) or df_name == '_':
                df_name = input('Dataframe not found in global variables. Please enter a name for the DataFrame: ')

        file_path = f'{directory}\\{df_name}.csv'

        print(f'Exproting {df_name} to CSV...\n@ {file_path}\n')
        df.to_csv(file_path, index=False)
        print(f'Successfully exported {df_name} to CSV')
    except Exception as e:
        print(f'Error exporting to CSV: {e}')

def query_data(schema, data):
    try:
        # Define the SQL query
        query = f'SELECT * FROM [{schema}].[{data}]'
        # Load data into DataFrame
        df = pd.read_sql(query, engine)
        print(f'Successfully imported {data}')
        return df
    except Exception as e:
        print(f'Error querying data: {e}')
        return pd.DataFrame()

def read_directory():
    directory = os.getcwd()
    files = os.listdir(os.getcwd())
    print(f"Your Current Directory is: {directory}")
    print("Files in: %s" % (files))

In [2]:
# Database credentials
db_host = 'LBHHLWSQL0001.lbhf.gov.uk'
db_port = '1433'
db_name = 'IA_ODS'

# Create the connection string for SQL Server using pyodbc with Windows Authentication
connection_string = f'mssql+pyodbc://@{db_host}:{db_port}/{db_name}?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes'

# Create the database engine
engine = create_engine(connection_string)

In [3]:
customer_columns = {
    'InteractionsKey':'interactionskey',
    'ETLTaskID':'etltaskid',
    'RowNumber':'rownumber',
    'GroupName':'groupname',
    'Type':'type',
    'Direction':'direction',
    'TimeQueued':'timequeued',
    'TimeEnded':'timeended',
    'Queue':'queue',
    'Skill':'skill',
    'Agent':'agent',
    'Username':'username',
    'UserID':'userid',
    'CustomerName':'customername',
    'CustomerContact':'customercontact',
    'QueueWait':'queuewait',
    'Duration':'duration',
    'TalkTime':'talktime',
    'CallHoldDuration':'callholdduration',
    'WrapUpDuration':'wrapupduration',
    'AgentHandlingTime':'agenthandlingtime',
    'Result':'result',
    'ActivityCode':'activitycode',
    'AssistedBy':'assistedby',
    'TransferredTo':'transferredto',
    'MediaInteractionId':'interactionid',
    'Subject':'subject',
    'CaseReference':'casereference',
    'Tags':'tags',
    'Forwarded':'forwarded',
    'IPAddress':'ipaddress',
    'FormattedSource':'formattedsource',
    'ConversationID':'conversationid',
    'ConversationStartTime':'conversationstarttime',
    'ConversationEndTime':'conversationendtime',
    'ConversationDuration':'conversationduration'
}
master_columns = {
    'Interaction Id':'interactionid',
    'Type':'type',
    'Direction':'direction',
    'Time Queued':'timequeued',
    'Time Ended':'timeended',
    'Queue':'queue',
    'Skill':'skill',
    'Agent':'agent',
    'Customer Name':'customername',
    'Customer Contact':'customercontact',
    'Queue Time':'queuetime',
    'Duration':'duration',
    'Talk Time':'talktime',
    'Hold Time':'holdtime',
    'Wrap-up Duration':'wrap-upduration',
    'Agent Handling Time':'agenthandlingtime',
    'Result':'result',
    'Activity Code':'activitycode',
    'Assisted By':'assistedby',
    'Transferred To':'transferredto',
    'Subject':'subject',
    'Conversation Start Time':'conversationstarttime',
    'Conversation End Time':'conversationendtime',
    'Conversation Duration':'conversationduration',
    'Agent Username':'agentusername',
    'Agent ID':'agentid',
    'Case Reference':'casereference',
    'Forwarded':'forwarded',
    'Source':'source',
    'Conversation ID':'conversationid',
    'IP Address':'ipaddress'
}

In [4]:
customer_service_data = query_data(schema='Netcall', data='Interactions')

Successfully imported Interactions


In [6]:
root_dir = r"C:\Users\jf79\OneDrive - Office Shared Service\BI - Corporate - Interactions Data"
os.chdir(root_dir)

read_directory()

# Create an empty DataFrame to store the combined data
master_df = pd.DataFrame()
 
# Iterate over each folder in the root directory
for folder in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder)
    # Ensure it's a directory
    if os.path.isdir(folder_path):
        # Iterate over each CSV file in the folder
        for file in os.listdir(folder_path):
            if file.endswith(".csv"):  # Ensure it's a CSV file
                file_path = os.path.join(folder_path, file)
 
                # Read CSV
                df = pd.read_csv(file_path)
 
                # Add a new column with the CSV filename
                df["Source_File"] = file
 
                # Append to master DataFrame
                master_data = pd.concat([master_data, df], ignore_index=True)

Your Current Directory is: C:\Users\jf79\OneDrive - Office Shared Service\BI - Corporate - Interactions Data
Files in: ['desktop.ini', 'Interactions Feb 2023 - Apr 2024 (excl Cust Serv)', 'Interactions Jan 2022 - Dec 2022 (excl Cust Serv)', 'Interactions Jan 2024 - Jun 2024 (excl Cust Serv)', 'Interactions Jun 24 - Jan 25 (excl Cust Serv)', 'Interactions_Jan 2023 (LIMITED GROUPS)']


In [7]:
customer_service = customer_service_data.copy()
customer_service.rename(
    columns=customer_columns,
    inplace=True
)
columns_to_drop = [
    'interactionskey','etltaskid','rownumber',
    'username','userid','queuewait','callholdduration',
    'wrapupduration','tags','formattedsource'
]
customer_service.drop(columns=columns_to_drop, inplace=True)
col = customer_service.pop('interactionid')
customer_service.insert(0, 'interactionid', col)
customer_service['groupname'] = customer_service['groupname'].str.replace('Customer_Services','Customer Services')

customer_service.sort_values(
    ['timequeued'],
    ascending=True,
    inplace=True
)

drop = customer_service[customer_service['interactionid'].duplicated(keep='first')]
customer_service.drop(
    drop.index,
    inplace=True
)
customer_service.reset_index(
    drop='index',
    inplace=True
)

customer_service['timequeued'] = pd.to_datetime(customer_service['timequeued'], format='mixed', dayfirst=True)
customer_service['timeended'] = pd.to_datetime(customer_service['timeended'], format='mixed', dayfirst=True)
customer_service['Year'] = customer_service['timequeued'].dt.year
customer_service['Month'] = customer_service['timequeued'].dt.month

In [8]:
master_data

,Type,Direction,Time Queued,Time Ended,Group,Queue,Skill,Agent,Customer Name,Customer Contact,...,Conversation Duration,Agent Username,Agent ID,Case Reference,Forwarded,Source,Conversation ID,Tags,IP Address,Source_File
0,Call,Inbound,30/12/2022 12:21,30/12/2022 12:21,Ace and Admissions,ACE and Admissions,ACE & Admissions,Lisa Border,NaN,7376392999,...,00:00:07,677.0,6438.0,NaN,NaN,Dialogue,4199429,NaN,NaN,ACE and Admissions.csv
1,Call,Inbound,30/12/2022 12:06,30/12/2022 12:07,Ace and Admissions,ACE and Admissions,ACE & Admissions,Snjezana Burchell,NaN,Unknown,...,00:00:15,2804.0,2804.0,NaN,NaN,Dialogue,4199385,NaN,NaN,ACE and Admissions.csv
2,Call,Inbound,30/12/2022 12:02,30/12/2022 12:05,Ace and Admissions,ACE and Admissions,ACE & Admissions,Lisa Border,NaN,Unknown,...,00:03:00,677.0,6438.0,NaN,NaN,Dialogue,4199373,NaN,NaN,ACE and Admissions.csv
3,Call,Inbound,30/12/2022 12:01,30/12/2022 12:01,Ace and Admissions,ACE and Admissions,ACE & Admissions,Snjezana Burchell,NaN,7940039730,...,00:00:15,2804.0,2804.0,NaN,NaN,Dialogue,4199367,NaN,NaN,ACE and Admissions.csv
4,Call,Inbound,30/12/2022 11:57,30/12/2022 12:00,Ace and Admissions,ACE and Admissions,ACE & Admissions,Lisa Border,NaN,7940039730,...,00:03:04,677.0,6438.0,NaN,NaN,Dialogue,4199347,NaN,NaN,ACE and Admissions.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1577735,Call,Inbound,11/01/2023 16:55:45,11/01/2023 16:55:46,NaN,Asylum Seekers or Refugee,Asylum Seekers or Refugee,NaN,NaN,7498214134,...,00:00:01,NaN,NaN,NaN,NaN,Dialogue,4248181,NaN,NaN,Refugee Team.csv
1577736,Call,Inbound,09/01/2023 11:32:39,09/01/2023 11:32:57,NaN,Anything Else,Anything Else,Kaideon Palma,NaN,7881864090,...,00:00:18,5650.0,5650.0,NaN,NaN,Dialogue,4231433,NaN,NaN,Refugee Team.csv
1577737,Call,Inbound,09/01/2023 11:30:53,09/01/2023 11:31:07,NaN,Ukrainian Friends And Family Scheme,Ukrainian Friends And Family Scheme,NaN,NaN,7881864090,...,00:00:14,NaN,NaN,NaN,NaN,Dialogue,4231409,NaN,NaN,Refugee Team.csv
1577738,Call,Inbound,09/01/2023 11:24:31,09/01/2023 11:24:56,NaN,Ukrainian Friends And Family Scheme,Ukrainian Friends And Family Scheme,Isak Bashir,NaN,7881864090,...,00:00:25,1697.0,1697.0,NaN,NaN,Dialogue,4231301,NaN,NaN,Refugee Team.csv


In [ ]:
master_df = master_data.copy()
master_df.rename(
    columns=master_columns,
    inplace=True
)

# Step 1: Split at the first dot (.)
master_df[['groupname', 'to_drop']] = master_df['Source_File'].str.split('.', expand=True)
 
# Step 2: Split at the first opening parenthesis (()
master_df[['groupname', 'to_drop']] = master_df['groupname'].str.split('(', expand=True)
 
# Step 3: Clean text (remove punctuation & normalize spaces)
master_df['groupname'] = master_df['groupname'].apply(lambda text: re.sub(r"[^\w\s]", "", text).strip())
master_df['groupname'] = master_df['groupname'].str.replace('Childrens', 'Children')
master_df['groupname'] = master_df['groupname'].str.replace('Children', 'Childrens')
master_df['groupname'] = master_df['groupname'].str.replace('ICAT6600', 'ICAT 6600')
master_df['groupname'] = master_df['groupname'].str.replace('Refugee', 'Refuge')
master_df['groupname'] = master_df['groupname'].str.replace('Refuge', 'Refugee')
master_df['groupname'] = master_df['groupname'].str.replace('Netcall HMS', 'HMS')
master_df['groupname'] = master_df['groupname'].str.replace('HMS', 'Netcall HMS')
master_df['groupname'] = master_df['groupname'].str.replace('Recovery Queue', 'Recovery')

col = master_df.pop('interactionid')
master_df.insert(0, 'interactionid', col)
col = master_df.pop('groupname')
master_df.insert(1, 'groupname', col)

master_df.sort_values(
    ['timequeued','agentid'],
    ascending=True,
    inplace=True
)

master_df.drop(
    columns=['Tags','Source_File','to_drop','Group'],
    inplace=True
)
drop = master_df[master_df['interactionid'].duplicated(keep='first')]
master_df.drop(
    drop.index,
    inplace=True
)
master_df.reset_index(
    drop='index',
    inplace=True
)


master_df['timequeued'] = pd.to_datetime(master_df['timequeued'], format='mixed', dayfirst=True)
master_df['timeended'] = pd.to_datetime(master_df['timeended'], format='mixed', dayfirst=True)
master_df['Year'] = master_df['timequeued'].dt.year
master_df['Month'] = master_df['timequeued'].dt.month

In [15]:
pd.set_option('display.max_columns', None)
final_df = pd.concat([master_df,customer_service])
validate_data(final_df)

#########################################################################################################################################################################################
DataFrame: final_df


,interactionid,groupname,type,direction,timequeued,timeended,queue,skill,agent,customername,customercontact,queuetime,duration,talktime,holdtime,wrap-upduration,agenthandlingtime,result,activitycode,assistedby,transferredto,subject,conversationstarttime,conversationendtime,conversationduration,agentusername,agentid,casereference,forwarded,source,conversationid,ipaddress,Year,Month
0,2640671,Repair CSC,Email,Inbound,2022-01-01 02:15:00,2022-01-04 16:01:00,Repairs customer e-mail,E-Mail Responses - Customer,Georgia St Rose,NaN,Gbenga.Osunkoya@lbhf.gov.uk,85:46:29,00:04:05,00:04:05,00:00:00,00:00:04,00:04:09,Replied,Repair Status,NaN,NaN,RE: Inspection item Requires Attention - (BLK)...,01/01/2022 02:15,04/01/2022 16:01,85:46:28,4861,4861.0,NaN,0.0,Email,2516337,NaN,2022,1
1,2640679,Repair CSC,Email,Inbound,2022-01-01 06:14:00,2022-01-04 16:05:00,Repairs Internal e-mail,E-mail Responses - Internal,Georgia St Rose,NaN,Baboucarr.Gaye@lbhf.gov.uk,81:51:19,00:03:35,00:03:34,00:00:01,00:00:09,00:03:43,Replied,Repair Status,NaN,NaN,RE: Norland House Electrical Failure (Job Ref....,01/01/2022 06:14,04/01/2022 16:05,81:51:20,4861,4861.0,NaN,0.0,Email,2516345,NaN,2022,1
2,2640681,Repair CSC,Email,Inbound,2022-01-01 09:08:00,2022-01-04 16:05:00,Repairs Internal e-mail,E-mail Responses - Internal,Emma Hicks,NaN,orbisreports@orbisprotect.com,78:56:42,00:03:06,00:03:06,00:00:00,00:00:16,00:03:22,Transferred,OOH - Related,NaN,Mehul Patel,[Orbis Protect] - Unallocated Cases,01/01/2022 09:08,04/01/2022 16:17,79:09:21,971,971.0,NaN,0.0,Email,2516347,NaN,2022,1
3,2640683,Repair CSC,Email,Inbound,2022-01-01 09:38:00,2022-01-04 16:19:00,Repairs customer e-mail,E-Mail Responses - Customer,Emma Hicks,NaN,noreply@lbhf.gov.uk,78:41:00,00:14:11,00:14:10,00:00:01,00:01:00,00:15:10,Replied,Repair Status,NaN,NaN,Form submission from: Housing repair notificat...,01/01/2022 09:38,04/01/2022 16:19,78:41:00,971,971.0,NaN,0.0,Email,2516349,NaN,2022,1
4,2640689,Repair CSC,Email,Inbound,2022-01-01 11:49:00,2022-01-04 16:11:00,Repairs customer e-mail,E-Mail Responses - Customer,Georgia St Rose,NaN,photobook@housemark.co.uk,76:21:50,00:05:42,00:05:42,00:00:00,00:00:03,00:05:45,Replied,Report New Repair,NaN,NaN,Inspection item Requires Attention - (BLK) 1-1...,01/01/2022 11:49,04/01/2022 16:11,76:21:50,4861,4861.0,NaN,0.0,Email,2516355,NaN,2022,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958538,7497727,Customer Services,Call,Inbound,2024-12-31 16:33:40,2024-12-31 16:35:53,Switchboard,Switchboard,Santiago Dibos,,07934623322,NaN,00:02:00,00:02:00,NaN,NaN,00:02:10,Answered,,,,,31/12/2024 16:33:40,31/12/2024 16:35:53,00:02:13,NaN,NaN,,N/A,NaN,7135141,,2024,12
958539,7497733,Customer Services,Call,Inbound,2024-12-31 16:38:39,2024-12-31 16:38:43,Switchboard,Switchboard,Anne Baxter,,07717152665,NaN,00:00:00,00:00:00,NaN,NaN,00:00:00,Abandoned,,,,,31/12/2024 16:38:39,31/12/2024 16:38:43,00:00:04,NaN,NaN,,N/A,NaN,7135147,,2024,12
958540,7497735,Customer Services,Call,Outbound,2024-12-31 16:41:10,2024-12-31 16:44:48,Environment,Environment Voice,Jennifer Alexander,,07400793639,NaN,00:03:28,00:03:28,NaN,NaN,00:03:38,Answered,,,,,31/12/2024 16:41:10,31/12/2024 16:44:48,00:03:38,NaN,NaN,,N/A,NaN,7135149,,2024,12
958541,7497737,Customer Services,Call,Inbound,2024-12-31 16:49:20,2024-12-31 16:59:38,Law Enforcement Team,Law Enforcement Team,Irum Kasmani,,07351593557,NaN,00:10:04,00:10:04,NaN,NaN,00:10:14,Answered,Other,,,,31/12/2024 16:49:20,31/12/2024 16:59:38,00:10:18,NaN,NaN,,N/A,NaN,7135151,,2024,12


Unique values per field:


,Field Name,No. of Unique Values
0,interactionid,2407735
1,groupname,48
2,type,3
3,direction,2
4,timequeued,1910390
5,timeended,1917116
6,queue,186
7,skill,199
8,agent,761
9,customername,736



Number of duplicate rows:
0 

<class 'pandas.core.frame.DataFrame'>
Index: 2407735 entries, 0 to 958542
Data columns (total 34 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   interactionid          2407735 non-null  object        
 1   groupname              2407735 non-null  object        
 2   type                   2407735 non-null  object        
 3   direction              2407735 non-null  object        
 4   timequeued             2407735 non-null  datetime64[ns]
 5   timeended              2407735 non-null  datetime64[ns]
 6   queue                  2407735 non-null  object        
 7   skill                  2206292 non-null  object        
 8   agent                  2166798 non-null  object        
 9   customername           976809 non-null   object        
 10  customercontact        2401959 non-null  object        
 11  queuetime              1223777 non-null  object        
 12  dur

None


Summary statistics:


,timequeued,timeended,Year,Month
count,2407735,2407735,2.407735e+06,2.407735e+06
mean,2023-07-21 15:56:18.172871680,2023-07-21 23:43:24.814285824,2.023076e+03,6.236796e+00
min,2022-01-01 02:15:00,2022-01-04 07:40:00,2.022000e+03,1.000000e+00
25%,2022-10-03 15:43:14.500000,2022-10-04 09:39:00,2.022000e+03,3.000000e+00
50%,2023-07-28 15:30:04,2023-07-28 17:00:16,2.023000e+03,6.000000e+00
75%,2024-05-02 14:42:32.500000,2024-05-03 09:19:05.500000,2.024000e+03,9.000000e+00
max,2025-02-12 17:40:17,2025-02-12 17:41:10,2.025000e+03,1.200000e+01
std,NaN,NaN,8.855617e-01,3.423327e+00


End of data validation
#########################################################################################################################################################################################



In [16]:
aggregated_interactions = final_df.groupby(['direction','type','Year','Month','groupname','queue']).agg(
    number_of_interactions = ('interactionid', 'count')
).reset_index()

export_to_csv(aggregated_interactions)

Exproting aggregated_interactions to CSV...
@ C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Python CSV Repositry\aggregated_interactions.csv

Successfully exported aggregated_interactions to CSV
